In [1]:
import collections

# data initialize
transition_dict = collections.defaultdict(dict)
transition_key = ['start','hot','cold','end']
transition_arr = [[0.0, 0.8, 0.2, 0.0],[0.0, 0.6, 0.3, 0.1],[0.0, 0.4, 0.5, 0.1],[0.0, 0.0, 0.0, 0.0]]
for i in range(len(transition_key)):
    for j in range(len(transition_key)):
        transition_dict[transition_key[j]][transition_key[i]]=transition_arr[i][j]

emission_dict = collections.defaultdict(dict)
emission_key = ['hot','cold']
emission_arr = [[0.0,0.0],[0.2,0.5],[0.4,0.4],[0.4,0.1]]

observation = [3,1,3]
for i in range(len(emission_arr)):
    for j in range(len(emission_key)):
        emission_dict[i][emission_key[j]]=emission_arr[i][j]

In [ ]:
print(transition_dict['hot']['start'],emission_dict[3]['hot'],transition_dict['hot']['hot'],emission_dict[1]['hot'],transition_dict['cold']['hot'],emission_dict[3]['cold'])

In [2]:
def forward(start_keyword:str, transition_dict:collections.defaultdict(dict), emission_key:list, observation_list:list) -> collections.defaultdict(dict):
    # dp[상태][관측값(T)]
    dp = collections.defaultdict(dict)

    # start로부터 첫번째 dp를 결정짓기 위함 (initialize)
    for i in emission_key:
        dp[i][0] = transition_dict[i][start_keyword]*emission_dict[observation_list[0]][i]


    for observ in range(1,len(observation_list)):
        for to_emi in emission_key:
            dp[to_emi][observ]=0
            for from_emi in emission_key:
                # print(from_emi,to_emi)
                dp[to_emi][observ] = (dp[from_emi][observ-1]*transition_dict[to_emi][from_emi]*emission_dict[observation_list[observ]][to_emi]) + dp[to_emi][observ]
    
    # end로의 전방확률구하기
    # for i in emission_key:
    #     dp[i][(list(dp[i].keys())[-1])+1] = transition_dict['end'][i]*dp[i][(list(dp[i].keys())[-1])]
    return dp

In [3]:
forward('start',transition_dict,emission_key,observation)

defaultdict(dict,
            {'cold': {0: 0.020000000000000004,
              1: 0.053000000000000005,
              2: 0.003850000000000001},
             'hot': {0: 0.32000000000000006,
              1: 0.04000000000000001,
              2: 0.018080000000000006}})